In [1]:
import torch
from torch import nn
from torch import optim
import numpy as np
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from loader_util.datasets import CustomTorchDataset, train_test_split_paths 
from torchvision import datasets, transforms, models

In [ ]:
# define constants
batch_size = 32
num_epochs = 10

In [ ]:
data_dir = r"/home/mhasan3/Desktop/WorkFolder/cellImages3/"

In [ ]:
baseModel = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\mhasa/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [ ]:
# remove classifier head
for param in baseModel.parameters():
    param.requires_grad = False

In [ ]:
# remove head
cell_types = 3
fcHead = nn.Linear(in_features=4096, out_features=cell_types)
baseModel.classifier[6] = fcHead

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
trainPaths, testPaths = train_test_split_paths(data_dir)

In [ ]:
train_data = CustomTorchDataset(pathList=trainPaths,
                                tranforms=train_transform)
test_data = CustomTorchDataset(pathList=testPaths,
                                tranforms=valid_transforms)

In [ ]:
train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)
test_loader = DataLoader(test_data,
                         batch_size=batch_size)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
baseModel.to(device=device)
device

In [ ]:
params_to_update = []
print("Params to learn:")

for name,param in baseModel.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
epoch_train_losses = []
epoch_train_accus = []
epoch_valid_losses = []
epoch_valid_accus = []

# define the loss
criterion = nn.CrossEntropyLoss()

# create the optimizer
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# loop over the epochs
for epoch in range (1, num_epochs + 1):
    
    # keep track of all losses
    running_train_loss = 0
    running_valid_loss = 0
    
    running_train_acc = 0
    running_valid_acc = 0
    
    train_batch_no = 0
    valid_batch_no = 0
    
    # TRAIN
    baseModel.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        train_batch_no += 1
        
        # Training pass
        optimizer.zero_grad()
        logits = baseModel(images)
        loss = criterion(logits, labels)
        
        # exponentiate the logits
        exp_logits = torch.exp(logits)
        
        # pass thru softmax to calculate probs
        probs = exp_logits / (torch.sum(exp_logits, dim=1).view(exp_logits.shape[0],-1))
        
        # get top class and to class prob
        top_p, top_class = probs.topk(1, dim=1)
        
        # reshape labels into column vector
        labels = labels.view(labels.shape[0], -1)
    
        # find which labels are correctly classified
        acc_boolean = top_class == labels
       
        # finally calculate accuracy
        running_train_acc += np.sum(acc_boolean.cpu().numpy()) / len(acc_boolean)
        
        # make gradient descent step
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
    else:
        epoch_train_loss = running_train_loss / train_batch_no
        epoch_train_losses.append(epoch_train_loss)
        epoch_train_accu = running_train_acc / train_batch_no
        epoch_train_accus.append(epoch_train_accu)
        print(f"Epoch: {epoch}")
        print(f'{train_batch_no} train batches completed')
        
        
    
    # VALIDATE
    baseModel.eval()
    with torch.no_grad():
        for valid_images, valid_labels in test_loader:
            valid_images, valid_labels = valid_images.to(device), valid_labels.to\
                (device)
            valid_batch_no += 1
            
            # Validation pass
            valid_logits = baseModel(valid_images)
            valid_loss = criterion(valid_logits, valid_labels)
            
            # exponentiate the logits
            valid_exp_logits = torch.exp(valid_logits)
            
            # pass thru softmax to calculate probs
            valid_probs = valid_exp_logits / (torch.sum(valid_exp_logits, dim=1)
                                              .view(valid_exp_logits.shape[0],-1))
            
            # get top class and to class prob
            valid_top_p, valid_top_class = valid_probs.topk(1, dim=1)
            
            # reshape labels into column vector
            valid_labels = valid_labels.view(valid_labels.shape[0], -1)
        
            # find which labels are correctly classified
            valid_acc_boolean = valid_top_class == valid_labels
           
            # finally calculate accuracy
            running_valid_acc += np.sum(valid_acc_boolean.cpu().numpy()) / len\
                (valid_acc_boolean)
    
            running_valid_loss += valid_loss.item()
        else:
            # dividing because running loss is the sum loss of all batches 
            epoch_valid_loss = running_valid_loss / valid_batch_no
            epoch_valid_accu = running_valid_acc / valid_batch_no
            epoch_valid_losses.append(epoch_valid_loss)
            epoch_valid_accus.append(epoch_valid_accu)
            print ( f"Training loss: {epoch_train_loss} " )
            print(f'Train accu: {epoch_train_accu}')
            print(f'{valid_batch_no} test batches completed')
            print ( f"Validation loss: {epoch_valid_loss}")
            print(f'Validation accu: {epoch_valid_accu}')
            print("=" * 50)

In [ ]:
# evaluate the network
# plot the performance
import pandas as pd
epochs = range(1, num_epochs+1)
plot_df = pd.DataFrame(data=np.c_[epochs, epoch_train_losses, 
                                  epoch_valid_losses, epoch_train_accus, 
                                  epoch_valid_accus], 
                       columns=['epochs','train_loss', 'test_loss', 
                                'train_acc', 'valid_acc'])

# do the actual plots
sns.set(font_scale=1)
f, ax = plt.subplots(1, 1, figsize=(15,8))
sns.lineplot(data=plot_df, x='epochs', y='train_loss', ax=ax, label='trainloss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='test_loss', ax=ax, label='val loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='valid_acc', ax=ax, label='val_acc', 
             linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='train_acc', ax=ax, 
             label='train_acc', linewidth=3)
ax.set_ylabel('Loss/Accuracy')
ax.set_xlabel('Epochs')
plt.setp(ax.get_legend().get_texts(), fontsize='18'); # for legend text         

#### INFERENCE

In [ ]:
for param_tensor in baseModel.state_dict():
    print(param_tensor, "\t", baseModel.state_dict()[param_tensor].size())

In [ ]:
torch.save(baseModel, 'torch_checkpoint.pth')

In [ ]:
model = torch.load('torch_checkpoint.pth')
model.eval()

In [ ]:
import cv2
image_name = 'MG63Sample.jpg'
cv_img = cv2.imread(image_name)
plt.imshow(cv_img[:,:,::-1])

In [ ]:
pil_img = Image.open(image_name)

In [ ]:
transformed_pil_image = valid_transforms(pil_img)

In [ ]:
transformed_pil_image.shape

In [ ]:
transformed_pil_image = torch.unsqueeze(transformed_pil_image, 0)
transformed_pil_image.shape

In [ ]:
logits_for_image = model(transformed_pil_image.to(device))

In [ ]:
exp_logits = torch.exp(logits_for_image)

In [ ]:
probs = exp_logits / torch.sum(exp_logits)
probs

In [ ]:
train_data.classes

In [ ]:
i = Image.fromarray(cv_img[:,:,::-1])

In [ ]:
images, labels = next(iter(train_loader))

In [ ]:
labels

In [ ]:
logits.shape

In [ ]:
logits